In [1]:
import numpy as np
import torch 
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers.utils.quantization_config import BitsAndBytesConfig

from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType
from peft import TTLoraConfig

In [2]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,device_map="auto")


lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.QUESTION_ANS
)
print(lora_config)
# model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_config)

# model.print_trainable_parameters()

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.QUESTION_ANS: 'QUESTION_ANS'>, inference_mode=False, r=16, target_modules={'v', 'q'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


In [3]:
# tt_lora_config = TTLoraConfig(
#  r=16,
#  tt_shape =[64,16,9,64],
#  lora_alpha=32,
#  target_modules=["q", "v"],
#  lora_dropout=0.05,
#  bias="none",
#  task_type=TaskType.QUESTION_ANS
# )

tt_lora_config = TTLoraConfig(
    r=16,
    tt_shape=[64, 16, 9, 64],
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.QUESTION_ANS,
)
print(tt_lora_config)

model = get_peft_model(model, tt_lora_config)

TTLoraConfig(peft_type=<PeftType.TTLORA: 'TTLORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.QUESTION_ANS: 'QUESTION_ANS'>, inference_mode=False, r=16, tt_shape=[64, 16, 9, 64], target_modules={'v', 'q'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, layer_replication=None)


In [6]:
model

PeftModelForQuestionAnswering(
  (base_model): TTLoraModel(
    (model): PeftModelForQuestionAnswering(
      (base_model): LoraModel(
        (model): T5ForConditionalGeneration(
          (shared): Embedding(32128, 768)
          (encoder): T5Stack(
            (embed_tokens): Embedding(32128, 768)
            (block): ModuleList(
              (0): T5Block(
                (layer): ModuleList(
                  (0): T5LayerSelfAttention(
                    (SelfAttention): T5Attention(
                      (q): lora.Linear(
                        (base_layer): lora.Linear(
                          (base_layer): Linear(in_features=768, out_features=768, bias=False)
                          (lora_dropout): ModuleDict(
                            (default): Dropout(p=0.05, inplace=False)
                          )
                          (lora_A): ModuleDict(
                            (default): Linear(in_features=768, out_features=16, bias=False)
                          )
